In [1]:
library(INLA)
library(rSPDE)
library(readr)
library(fmesher)
library(inlabru)
data_ori <- as.data.frame(read_csv("tmp/1180_vol.csv"))
logpt<-log(data_ori$Close)
logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
date <- data_ori$Date[1:(length(data_ori$Date)-1)]
   
date_timeindex<-as.numeric(as.POSIXct(date)) 
date_timeindex <- (date_timeindex - min(date_timeindex))/86400
mesh <- fm_mesh_1d(date_timeindex)
data = list(logreturn = logreturn, times = date_timeindex)





Loading required package: Matrix



Loading required package: sp

This is INLA_24.04.01 built 2024-04-01 19:43:22 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation

This is rSPDE 2.3.3.9000
- See https://davidbolin.github.io/rSPDE for vignettes and manuals.

Rows: 2372 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (9): Open, High, Low, Close, Volume, Dividends, Stock Splits, vol10, vol30
date (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
build_index <- function(data_size, window) {
    train <- test <- list()

    k = 1
    for (i in seq(from = window, to = data_size - window, by = window)) {
        train[[k]] <- 1:i
        test[[k]] <- (i + 1):(i + window)
        k = k + 1
    }
    index = list(train = train, test = test)
    return(index)
}


In [5]:
exp_id=20
library(glue)
config_id<-29
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvol<-results$fit[[run_id+1]]$fit

config_id<-65
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvol.t<-results$fit[[run_id+1]]$fit

config_id<-45
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvol.nig<-results$fit[[run_id+1]]$fit



config_id<-110
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvolln<-results$fit[[run_id+1]]$fit

config_id<-150
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rs_stochvol<-results$fit[[run_id+1]]$fit

In [12]:
models=list(rough_stochvol,rough_stochvol.t,rough_stochvol.nig,rough_stochvolln,rs_stochvol)

In [48]:
res2 <- cross_validation(models,train_test_indexes = build_index(length(date_timeindex), 22))

In [46]:
res2=as.data.frame(lapply(res2,as.numeric))

In [47]:
res2

Model,dss,mse,crps,scrps
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,-7.20750016,0.0002668893,-0.008511069,-1.0344050
NA,-7.06377000,0.0002673929,-0.008569718,-1.0186201
NA,-7.06952248,0.0002674422,-0.008569490,-1.0222109
NA,-0.01842509,0.0007006237,-0.240798081,0.7711322
NA,-7.19576062,0.0002675747,-0.008510083,-1.0341094
NA,NA,NA,NA,NA


In [16]:
install.packages("xtable")
library(xtable)


Installing package into ‘/home/jianj0c/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)



In [36]:
xtable(res2, caption = "Example Data Frame", label = "tab:example_df",digits=c(0,0,3,3,3,3))

,Model,dss,mse,crps,scrps
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Model 1,-7.20750016342735,0.000266889342456855,-0.00851106902919709,-1.0344049941245
2,Model 2,-7.06376999814561,0.00026739290758667,-0.00856971761657976,-1.0186200721771
3,Model 3,-7.06952248439858,0.000267442214123041,-0.00856949019251315,-1.02221090756634
4,Model 4,-0.0184250850052743,0.000700623746038587,-0.240798081333949,0.771132197066555
5,Model 5,-7.19576062231673,0.000267574690181063,-0.00851008299119441,-1.03410944448769
,Best,Model 1,Model 1,Model 4,Model 1


In [43]:
# Convert to LaTeX table
latex_table <- xtable(res2, caption = "Example Data Frame", label = "tab:example_df",igits=3)

# Print the LaTeX table
print.xtable(latex_table, float.placement = "H")



% latex table generated in R 4.3.3 by xtable 1.8-4 package
% Sun Apr 21 21:34:13 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlllll}
  \hline
 & Model & dss & mse & crps & scrps \\ 
  \hline
1 & Model 1 & -7.20750016342735 & 0.000266889342456855 & -0.00851106902919709 & -1.0344049941245 \\ 
  2 & Model 2 & -7.06376999814561 & 0.00026739290758667 & -0.00856971761657976 & -1.0186200721771 \\ 
  3 & Model 3 & -7.06952248439858 & 0.000267442214123041 & -0.00856949019251315 & -1.02221090756634 \\ 
  4 & Model 4 & -0.0184250850052743 & 0.000700623746038587 & -0.240798081333949 & 0.771132197066555 \\ 
  5 & Model 5 & -7.19576062231673 & 0.000267574690181063 & -0.00851008299119441 & -1.03410944448769 \\ 
   & Best & Model 1 & Model 1 & Model 4 & Model 1 \\ 
   \hline
\end{tabular}
\caption{Example Data Frame} 
\label{tab:example_df}
\end{table}
